# Dominic's Final Project - EMS Data & Injury - 12.11.23

The purpose of this analysis is to understand and attempt to find an answer to the question, "Have the number and type of injuries in the EMS field (x) affected the number of people pursuing a career in the field (y)?" 

I am pursuing this question and analysis because of the personal connection and purpose it has in my own life. I am currently a 3rd generation EMT pursuing my career and as an EMT (soon to be a firefighter), I understand the risks associated with the profession. However, I also understand the importance of the profession and the duty to serve the community and help others. Unfortunately, there are not as many people pursuing the career as there were many years ago. 

Thus, for the purpose of this analysis, my goal is to understand how enrollment in the EMS field has changed and if it correlates with the risk of injuries in the field. 

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from scipy import stats
import requests
import base64
import json
import urllib

The following data has been collected from the US Bureau of Labor Statistics and the CDC. It is valuable as these are government sources that do consistent research in the field of EMS. Government sources are valuable for several reasons including providing information in a neutral and unbiased manner, keeping official records of various activities, being accountable for providing accurate information for maintaining trust, and more. 

Nevertheless, it's essential to approach any information, including from government sources, with a critical mindset. Governments can sometimes be subject to pressure and the accuracy of information may vary across different agencies.

To better visualize the data, I took each of the sets, copied them into an Excel sheet, and modified them.

In [2]:
ems_projections = pd.read_csv('Employment Projections for EMTs and Paramedics, 2022-32.csv') 

In [3]:
ems_projections #Data collected from the US Bureau of Labor Statistics
                #https://www.bls.gov/ooh/healthcare/emts-and-paramedics.htm#tab-6

,Occupational Title,"Employment, 2022","Projected Employment, 2032","Change, 2022-32 (Numeric)","Change, 2022-32 (Percentage)"
0,Emergency Medical Technicians and Paramedics,"269,000","283,600","14,600",5
1,Emergency Medical Technicians,"170,700","180,000","9,300",5
2,Paramedics,"98,300","103,600","5,300",5


In [4]:
ems_outlook = pd.read_csv('EMS Employment Outlook.csv')

In [5]:
ems_outlook #Data collected from the US Bureau of Labor Statistics

,Type of Work,2022 Employment,Projected 2032 Employment,"Employment Percent Change, 2022-2032"
0,Healthcare and social assistance,178.3,187.7,5.3
1,Ambulatory healthcare services,126.8,134.5,6.0
2,Offices of physicians,3.2,3.3,2.9
3,Offices of other health practitioners,0.5,0.6,12.7
4,Offices of all other health practitioners,0.5,0.5,11.6
5,Outpatient care centers,3.5,4.2,18.3
6,Medical and diagnostic laboratories,0.3,0.3,8.6
7,Other ambulatory healthcare services,119.2,126.0,5.7
8,Ambulance services,117.2,123.7,5.5
9,All other ambulatory healthcare services,2.0,2.3,18.2


In [6]:
ems_injuries_total = pd.read_csv('EMS Injuries (Total).csv') #All following data collected from the CDC
                                                            #https://www.cdc.gov/niosh/topics/ems/data.html#print

In [7]:
ems_injuries_total

,Number of Injuries (Total)
0,"16,900"


In [8]:
ems_injuries_sex = pd.read_csv('EMS Demographics (Sex).csv')

In [9]:
ems_injuries_sex

,Sex,Total
0,Male,"9,900"
1,Female,"7,000"


In [10]:
ems_injuries_age = pd.read_csv('EMS Demographics (Age).csv')

In [11]:
ems_injuries_age

,Age Group (in yrs),Total (Age Group)
0,<25,"2,800"
1,25-34,"7,000"
2,35-44,"3,800"
3,>44,"3,330"


In [12]:
ems_injuries_bodypart = pd.read_csv('EMS Injury (Body Part).csv')

In [13]:
ems_injuries_bodypart

,Body Part Affected,Count (1)
0,"Leg, including foot","3,200"
1,"Upper trunk, inc. neck & shoulder","3,000"
2,Arm,"2,200"
3,Lower trunk,"2,200"
4,Hand & finger,"1,900"


In [14]:
ems_injuries_diagnosis = pd.read_csv('EMS Injury (Diagnosis).csv')

In [15]:
ems_injuries_diagnosis

,Diagnosis,Count (2)
0,Sprains & strains,"4,000"
1,Contusions/abrasions,"2,600"


In [16]:
ems_injuries_event = pd.read_csv('EMS Injury (event).csv')

In [30]:
ems_injuries_event

,Event,Count (3)
0,Overexertion & bodily reaction,"5,200"
1,Violence,"3,000"
2,Harmful exposures,"2,500"
3,Transportation incidents,"2,500"
4,Contact with objects & equipment,"2,100"
5,"Falls, slips, trips","1,600"


In [18]:
ems_injuries_source = pd.read_csv('EMS Injury (Source).csv')

In [19]:
ems_injuries_source

,Source,Count (4)
0,"Persons, plants, animals, & minerals","9,000"
1,Vehicles,"4,100"
2,"Tools, instruments, and equipment","1,800"


In the following section of my report, I will describe and begin to interpret the data. To understand my goal of how enrollment in the EMS field has changed and if it correlates with the risk of injuries in the field, I plan to use a bar plot and/or a pie plot to visualize the data. I am choosing a bar plot or pie plot because since I have multiple, separate sets of data, that are already intrepted (data that is not raw) I can use the data that has been organized to group the different data sets. My hope is to visualize how the data from these different sets might correlate.

In [ ]:
fig1, ax1 = plt.subplots()

ax1.pie(polid.values, labels = polid.index)

plt.show()

In [ ]:
fig2, ax2 = plt.subplots()

ax5.bar(x = [1, 2, 3], height = )

plt.show()

As this report ends, there are gaps and inadequacies, rendering it incomplete. I believe the data report faced challenges related to completeness, from the necessity to modify the data for better visualization. While I intended to improve the presentation and interpretation of the data, transforming it has introduced complexities that impact the overall ability to interpret the report. 

Nevertheless, I can make some interpretations based on the data that was collected, stored, and organized. For example, in the table "ems_projections," projected employment from 2022 is expected to grow 5% between now and 2032 for both EMTs and paramedics. By understanding this data, we may be able to conclude that the number and type of injuries in the EMS field may not impact the number of people pursuing a career in the field.

The ability to collect data from the CDC about the injuries to EMS workers was a valuable data set. While it showed the number of injuries, gender, age, type of injury, the event that occurred, and how it occurred, it was difficult to use each of the data frames to construct a graph, chat, plot, or diagram to completely find an answer to my question. 

If I had to make an estimate, I would say that the number and type of injuries that occur in the field of EMS somewhat but not fully, relate to the number of people pursuing a career in EMS. I would guesstimate this because no one wants to get injured in their job/line of work. This may have impacted some people's decision to choose EMS as a career because they fear injury. This is a very logical concept. However, the field of health care and EMS is also rewarding. The ability to help people and positively impact their lives, despite your own, may be enough for some people to overcome their fears and step into the line of service. 

If I were to return to this data report in the future, I think there would be quite a few things I would change because of the limitations I had. I strongly believe that my question and goal for the project were one of value. Attempting to understand if, how, and why injuries in the EMS field relate to the number of people pursuing a career in the field, is important because we always need more caregivers in the world. With the way our world is today, having more people to serve the community can impact our world and keep people together.

However, because of the limitations in my data, I think that I would either need to find more sources of data or find a way to better organize the data. I think that because I had so many separate sheets, it affected my ability to parse through the data and find sets that work correctly with each other. I also think that if I approached the question from a different perspective, I may have been able to find data that would better answer my question. In other words, while I believe my question is purposeful, perhaps gathering data from periods like 1980, 1990, etc., and comparing it to today would change my analysis. The field of EMS has come a long way since 1980 as awareness of injury, mental health, technology, and more have advanced. Therefore, if the number of people pursuing careers in EMS has decreased in recent years, why were they so high many years ago? 
I think a different approach to my question could be the key to understanding if injuries relate to the pursuit of the EMS or if and what other reasons there may be for the change in the profession.

In the end, if the audience were to take away anything from this data report, I believe it would be that while there is risk in the field of healthcare, the benefits outweigh the costs. While this statement may be bais as I am an EMT myself, I think we would all agree that our healthcare workers are some of the strongest people our world has. The ability to put themselves aside for the greater good is something I have admired and is just one of the reasons why I decided to become an EMT. Yes, there is risk in this field as shown by the injury data (CDC), but there are also still people pursuing careers in the field (Labor Statistics).